In [2]:
import requests
from bs4 import BeautifulStoneSoup
import re
import unicodedata
import pandas as pd

In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [4]:
response = requests.get(static_url)

In [9]:
soup = BeautifulStoneSoup(response.text)

C:\Users\snirt\AppData\Local\Temp\ipykernel_31704\3042023801.py:1: DeprecationWarning: The BeautifulStoneSoup class is deprecated. Instead of using it, pass features="xml" into the BeautifulSoup constructor.
  soup = BeautifulStoneSoup(response.text)


In [13]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

In [23]:
html_table = soup.find_all('table')

In [29]:
first_launch_table = html_table[2]
#print(first_launch_table)

In [33]:
table_cells =first_launch_table.find_all('th')

In [37]:
table_cells

[<th scope="col">Flight No.
 </th>,
 <th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
 </th>,
 <th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
 </th>,
 <th scope="col">Launch site
 </th>,
 <th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
 </th>,
 <th scope="col">Payload mass
 </th>,
 <th scope="col">Orbit
 </th>,
 <th scope="col">Customer
 </th>,
 <th scope="col">Launch<br/>outcome
 </th>,
 <th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">1
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">2
 </th>,
 <

In [57]:
columns = []
for row in table_cells:
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
    column_name=' '.join(row.contents)
    
    if not(column_name.strip().isdigit()):
        column_name = column_name.strip()
        columns.append(column_name)

In [59]:
column_names = [column for column in columns if len(column)>0]

In [61]:
column_names

['Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome']

In [236]:
launch_dict = dict.fromkeys(column_names)
del launch_dict['Date and time ( )']

In [237]:
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [238]:
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"): 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag = False
        if flag:
            row = rows.find_all('td')
            launch_dict['Flight No.'].append(flight_number)
            
            date_time_list = [data_time.strip() for data_time in list(row[0].strings)][0:2]
            date = date_time_list[0].strip(',')
            time = date_time_list[1]
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)
            
            bv = [booster_version for i,booster_version in enumerate(row[1].strings) if i%2==0][0:-1]
            if not(bv):
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)
            
            launch_dict['Launch site'].append(row[2].a.string)
            
            launch_dict['Payload'].append(row[3].a.string)
            
            mass=unicodedata.normalize("NFKD",row[4].text).strip()
            if mass:
                mass.find("kg")
                new_mass=mass[0:mass.find("kg")+2]
            else:
                new_mass=0
            launch_dict['Payload mass'].append(new_mass)
                
            launch_dict['Orbit'].append(row[5].a.string)
            
            
            if row[6].a:
                customer=row[6].a.string
            else:
                customer=None
            launch_dict['Customer'].append(customer)
            
            launch_dict['Launch outcome'].append(list(row[7].strings)[0])
            
            launch_dict['Booster landing'].append([i for i in row[8].strings][0])

In [242]:
data = pd.DataFrame({key:pd.Series(value) for key, value in launch_dict.items()})

In [244]:
data.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,"[F9 v1.0, B0003.1]",Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,"[F9 v1.0, B0004.1]",Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,"[F9 v1.0, B0005.1]",No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,"[F9 v1.0, B0006.1]",No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,"[F9 v1.0, B0007.1]",No attempt\n,1 March 2013,15:10


In [245]:
data.to_csv('../datasets/datasetscrape.csv')